In [ ]:
import pandas as pd
import numpy as np
import math
import h5py
from sklearn.model_selection import train_test_split
import joblib
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from functions import prepare_data

import tensorflow_io as tfio
import tensorflow as tf


In [ ]:
input_file='/eos/project/d/dshep/L1anomaly/QCD_preprocessed.h5'
bsm_file = '/eos/project/d/dshep/L1anomaly/BSM_preprocessed.h5'
output_file='/eos/project/d/dshep/L1anomaly/CMS_QCD_data_flatten.pickle'

In [ ]:
def prepare_data(input_file, input_bsm, output_file, events=20000000):
    # read QCD data
    with h5py.File(input_file, 'r') as h5f:
        # remove last feature, which is the type of particle
        data = h5f['full_data_cyl'][:events,:,:]
        #np.random.shuffle(data)
        #data = data[:events,:,:]
    
    # fit scaler to the full data
    pt_scaler = StandardScaler()
    data_target = np.copy(data)
    data_target[:,:,0] = pt_scaler.fit_transform(data_target[:,:,0])
    data_target[:,:,0] = np.multiply(data_target[:,:,0], np.not_equal(data[:,:,0],0))

    data = data.reshape((data.shape[0],57))
    data_target = data_target.reshape((data_target.shape[0],57))
    # define training, test and validation datasets
    X_train, X_test, Y_train, Y_test = train_test_split(data, data_target, test_size=0.5)
    del data, data_target
    
    # read BSM data
    bsm_data = []
    
    with h5py.File(input_bsm,'r') as h5f2:
        for key in h5f2.keys():
            bsm_file = h5f[key][:,:,:]
            bsm_file = bsm_file.reshape(bsm_file.shape[0],bsm_file.shape[1]*bsm_file.shape[2])
            bsm_data.append(bsm)
    
    bsm_scaled_data=[]
    for bsm in bsm_data:
        bsm = bsm.reshape(bsm.shape[0],19,3,1)
        bsm = np.squeeze(bsm, axis=-1)
        bsm_data_target = np.copy(bsm)
        bsm_data_target[:,:,0] = pt_scaler.transform(bsm_data_target[:,:,0])
        bsm_data_target[:,:,0] = np.multiply(bsm_data_target[:,:,0], np.not_equal(bsm[:,:,0],0))
        bsm_data_target.reshape(bsm_data_target.shape[0], bsm_data_target.shape[1]*bsm_data_target.shape[2])
        bsm_scaled_data.append(bsm_data_target)
    
    data = [X_train, Y_train, X_test, Y_test, bsm_data, bsm_scaled_data, pt_scaler]

    with open(output_file, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
events = 20000000

In [ ]:
# data = tfio.IODataset.from_hdf5(input_file, '/full_data_cyl')
# data = data.take(NUMSAMPLES).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
prepare_data(input_file, bsm_file, output_file, events=20000000)